# 将oneke_wrapper中常规推理更换为vllm推理并进行测试

In [1]:
import json
from oneke_wrapper import OneKEWrapper

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


安装了——ipywidgets-8.1.5 jupyterlab-widgets-3.0.13 widgetsnbextension-4.0.13

## 定义模型路径以及卡号

In [2]:
model_path = "./cache/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6"
lora_path = "./lora/llama3-8B-iepile-data2text-continue"

device = "cuda:2"

## 初始化类对象,加载模型

In [3]:
oneke_wrapper = OneKEWrapper(model_path, lora_path, device, False)

Initializing OneKEWrapper...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


## vllm推理单条测试

In [4]:
# 加载数据
data = json.load(open("./internal_data_type/data2text_test_all-v1.json"))

#选取第一条进行推理
oneke_wrapper.inference(data[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[{'table': {'header': ['时间',
    '人物',
    '国家',
    '装备名称',
    '装备类型',
    '装备数量',
    '组织机构',
    '地点',
    '任务',
    '行动',
    '事件'],
   'data': {'2': ['去年五月',
     '军方检察官',
     '台湾',
     '笔记本电脑',
     '导弹艇',
     '一台',
     '海军',
     '未明确',
     '未明确',
     '展开调查',
     '笔记本电脑丢失']}},
  'origin': {},
  'text': '一些消息人士透露，这两张地图中包含有关台湾海军舰艇布署的情况。如果发生战争，丢失这两张地图将使台湾海军在面对敌方时失去战斗力。此外，还有一台导弹艇上的笔记本电脑在去年五月丢失，军方检察官认真对此展开调查。',
  'verify': {'2': [{'v': '去年五月', 'span': '', 'cf': ''},
    {'v': None, 'span': '', 'cf': ''},
    {'v': None, 'span': '', 'cf': ''},
    {'v': '笔记本电脑', 'span': '', 'cf': ''},
    {'v': '导弹艇', 'span': '', 'cf': ''},
    {'v': '一台', 'span': '', 'cf': ''},
    {'v': '海军', 'span': '', 'cf': ''},
    {'v': None, 'span': '', 'cf': ''},
    {'v': None, 'span': '', 'cf': ''},
    {'v': '展开调查', 'span': '', 'cf': ''},
    {'v': '笔记本电脑丢失', 'span': '', 'cf': ''}]},
  'add_refs': []}]

## vllm推理全集测试，记录平均时间

In [5]:
# 所有数据计算平均时间
from tqdm import tqdm
oneke_wrapper.data_count = 0
oneke_wrapper.total_time = 0
results = []
for d in tqdm(data):
    results.append(oneke_wrapper.inference(d))
print(oneke_wrapper.total_time/oneke_wrapper.data_count)

 14%|█▍        | 300/2138 [19:45<2:48:56,  5.52s/it]

Expecting ',' delimiter: line 1 column 193 (char 192)
input:{'instruction': '你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。', 'schema': ['时间', '人物', '国家', '装备名称', '装备类型', '装备数量', '组织机构', '地点', '任务', '行动', '事件'], 'input': '一项合作研发的先进无人机系统——雀鹰系统，由以色列国防研究与发展局、以色列XTEND公司和美国国防部反恐技术支持办公室联合开发。该系统采用远程沉浸式界面，可有效保护特种部队免受各种入境空中威胁，特别是无人机的威胁。美军计划启动一项保护特种部队的计划，采用雀鹰无人机系统。'}, output:{"时间": [], "人物": [], "国家": [], "装备名称": ["雀鹰系统", "雀鹰无人机系统"], "装备类型": ["无人机系统"], "装备数量": [], "组织机构": ["['以色列国防研究与发展局", "['以色列XTEND公司", "['美国国防部反恐技术支持办公室"], "地点": [], "任务": [], "行动": [], "事件": []


100%|█████████▉| 2134/2138 [2:25:25<00:12,  3.05s/it]  

Expecting value: line 1 column 1 (char 0)
input:{'instruction': '你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。', 'schema': ['国别', '军种', '单位', '装备', '装备数量', '指挥员', '地点', '行动', '开始时间', '任务状态', '任务类型', '备注'], 'input': '7月13日，陆军第54集团军第127师在华中某训练基地进行夜间实弹射击演习，本次演习中，部队使用07式自行榴弹炮（代号PLZ-07）12辆，发射155毫米炮弹36发，成功摧毁夜间模拟敌方目标。7月14日，海军第一航空母舰支队的辽宁舰（航空母舰，舷号16）和第三驱逐舰支队的西安舰（052C驱逐舰，舷号153）共1280人，在黄海进行舰载机起降训练。此次训练中，歼-15舰载机12架次完成了起降任务。'}, output:


100%|██████████| 2138/2138 [2:25:39<00:00,  4.09s/it]

Expecting value: line 1 column 1 (char 0)
input:{'instruction': '你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。', 'schema': ['国别', '军种', '装备数量', '装备', '地点', '行动', '开始时间', '任务状态'], 'input': '7月21日，空军第十战斗机师的苏-30战斗机（代号Su-30MKK）8架，在东海上空进行空中拦截任务。此次任务中，苏-30战斗机成功拦截并驱离进入防空识别区的2架不明飞行器，确保了空域安全。7月22日，陆军第27集团军第79师在华东某训练基地进行联合实弹射击演习，本次演习中，部队使用PLZ-45自行榴弹炮（代号PLZ-45）18辆，发射155毫米炮弹54发，成功摧毁模拟敌方目标，演习效果显著。'}, output:
4.085767626929897


In [6]:
print(f"Average inference time: {oneke_wrapper.total_time/oneke_wrapper.data_count}")

Average inference time: 4.085767626929897


In [10]:
import torch
print(torch.cuda.memory_allocated(device=2))
memory_used = torch.cuda.memory_allocated(device=2) - oneke_wrapper.gpu_mem_before_load
memory_occupied = torch.cuda.memory_allocated(device=2) / oneke_wrapper.gpu_mem_max * 100
print(f"Memory used for inference: {memory_used / (1024 ** 2):.2f} MB")
print(f"Memory occupied after inference: {memory_occupied:.2f}%")

16756924928
Memory used for inference: 15980.65 MB
Memory occupied after inference: 65.89%


In [8]:
print(len(results))
json.dump(results, open("./results/llama.json",'w+'))

2138
